In [1]:
import pandas as pd
from regain.utils import flatten
import numpy as np

In [2]:
filename = "../regain/data/text/webkb-train-stemmed.txt"
train = pd.read_csv(filename, header=None, sep='\t', index_col=0)
train.columns = ['words']

In [4]:
words = np.unique(flatten([words.split(' ') for words in train.words.tolist() if isinstance(words, str)]))

In [5]:
X = pd.DataFrame([], columns=words)
y = []
for row in train.itertuples():
    if isinstance(row.words, str):
        series = pd.Series(dict(zip(*np.unique(row.words.split(' '), return_counts=True))), name=row.Index)
        X = X.append(series)
        y.append(row.Index)

X = X.fillna(0)
X.index = y

y = np.asarray(y)

In [7]:
words_to_discard = []
for yy in np.unique(y):
    words_to_discard += list(X[words[X[y==yy].sum(axis=0) == 0]].columns)

words_to_keep = list(set(X.columns) - set(words_to_discard))

In [37]:
X.sum(axis=0)

aaa             4.0
aaai          104.0
aarhu           3.0
aaron          15.0
abandon         4.0
abbadi          5.0
abbrevi         3.0
abc            13.0
abdel           6.0
abdelsalam      7.0
abduct          9.0
abelson        11.0
abet            3.0
abhinav         3.0
abil           47.0
abiteboul      10.0
ablex           5.0
abort           4.0
aboulnaga       4.0
abr             3.0
abraham        14.0
abram          11.0
abridg          9.0
abroad          8.0
absenc          7.0
absent          5.0
absolut        32.0
absorpt         3.0
abstract      314.0
abu             8.0
              ...  
zachari         4.0
zahorjan        6.0
zaki            3.0
zealand        17.0
zebra           6.0
zen             6.0
zena            3.0
zhai            3.0
zhang          45.0
zhao            7.0
zhejiang       13.0
zheng           7.0
zhihong         3.0
zhongshan       7.0
zhou           11.0
zhu             6.0
zip            15.0
zippel          7.0
zone           14.0


In [48]:
def logentropy_normalize(X):
    P = X / X.values.sum(axis=0, keepdims=True)
    E = 1 + (P * np.log(P)).fillna(0).values.sum(axis=0, keepdims=True) / np.log(X.shape[0] + 1)
    return E * np.log(1 + X)

In [49]:
X_new = logentropy_normalize(X)[words_to_keep]

/Users/federicot/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
X_new.T.sort_index().T

,aaron,abc,abil,absolut,abstract,academ,acceler,accept,access,accompani,...,yellow,yield,york,young,yue,zabih,zhang,zhou,zpl,zurich
student,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.341292,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
student,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
faculty,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
student,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
project,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
faculty,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
faculty,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.435808,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
faculty,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.209447,0.478335,...,0.0,0.000000,0.435808,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
student,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
course,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.274965,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [27]:
from gensim.models import LogEntropyModel
from gensim.test.utils import common_texts
from gensim.corpora import Dictionary

corp = [w.split(' ') for w in train.words.tolist() if isinstance(w, str)]
text = corp #common_texts
dct = Dictionary(text)  # fit dictionary
corpus = [dct.doc2bow(row) for row in text]  #convert to BoW format
model = LogEntropyModel(corpus, normalize=True)  # fit model

In [52]:
import gensim
XX = gensim.matutils.corpus2dense(model[corpus], num_terms=words.size).T
cols = list(dct.values())
df = pd.DataFrame(XX, columns=cols, index=y)
df[words_to_keep].T.sort_index().T

,aaron,abc,abil,absolut,abstract,academ,acceler,accept,access,accompani,...,yellow,yield,york,young,yue,zabih,zhang,zhou,zpl,zurich
student,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.056952,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
student,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
faculty,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
student,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
project,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
faculty,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
faculty,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.174050,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
faculty,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.040776,0.093125,...,0.0,0.000000,0.084845,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
student,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
course,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.063852,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [55]:
from regain.covariance import kernel_time_graphical_lasso_

In [58]:
mdl = kernel_time_graphical_lasso_.KernelTimeGraphicalLasso(kernel=np.ones((np.unique(y).size, np.unique(y).size)))

In [61]:
X = df[words_to_keep].values

In [64]:
mdl.fit(X, df.index)

KeyboardInterrupt: 